In [2]:
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.optim as optim
# For reproducibility
torch.manual_seed(1)

In [4]:
# df = pd.read_csv("/kaggle/input/2021-ai-w3-p1/train.csv")
train_df = pd.read_csv("./배추가격예측_data/train.csv")
test_df = pd.read_csv("./배추가격예측_data/test.csv")
submit_df = pd.read_csv("./배추가격예측_data/submit_sample.csv")
print(train_df.head())
print(test_df.head())
print(submit_df.head())

       year  avgTemp  minTemp  maxTemp  rainFall  avgPrice
0  20100101     -4.9    -11.0      0.9       0.0      2123
1  20100102     -3.1     -5.5      5.5       0.8      2123
2  20100103     -2.9     -6.9      1.4       0.0      2123
3  20100104     -1.8     -5.1      2.2       5.9      2020
4  20100105     -5.2     -8.7     -1.8       0.7      2060
       year  avgTemp  minTemp  maxTemp  rainFall
0  20170107      4.0     -1.4      9.5       0.1
1  20170120     -1.7     -5.4      1.4       3.3
2  20170223      1.4     -2.0      5.0       0.0
3  20170227      3.6     -2.5     11.1       0.1
4  20170309      5.0     -1.6     11.3       0.0
   Id   Expected
0   0  2997.8792
1   1  2777.1380
2   2  2868.9387
3   3  3113.0390
4   4  3039.6218


In [36]:
x_train = np.array([train_df['avgTemp'],train_df['minTemp'],train_df['maxTemp'],train_df['rainFall']])
x_train = x_train.T
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(train_df['avgPrice'])
print(x_train.shape)
print(y_train.shape)

torch.Size([2557, 4])
torch.Size([2557])


In [38]:
# 모델 초기화
w = torch.zeros((4,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w, b], lr=1e-5)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(w) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} w:{} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w, b.item(), cost.item()
        ))

Epoch    0/10000 w:tensor([[0.8280],
        [0.5309],
        [1.1705],
        [0.2273]], requires_grad=True) b: 0.064 Cost: 12013343.000000
Epoch  100/10000 w:tensor([[38.5997],
        [18.9156],
        [60.9042],
        [10.6542]], requires_grad=True) b: 4.136 Cost: 5126071.000000
Epoch  200/10000 w:tensor([[43.5992],
        [12.3714],
        [78.5713],
        [12.2778]], requires_grad=True) b: 6.470 Cost: 4694184.500000
Epoch  300/10000 w:tensor([[42.9296],
        [ 1.9272],
        [88.5235],
        [12.5497]], requires_grad=True) b: 8.439 Cost: 4480125.000000
Epoch  400/10000 w:tensor([[41.3259],
        [-8.7644],
        [96.7853],
        [12.7394]], requires_grad=True) b: 10.286 Cost: 4291323.000000
Epoch  500/10000 w:tensor([[ 39.6146],
        [-19.0860],
        [104.4289],
        [ 13.0407]], requires_grad=True) b: 12.055 Cost: 4120052.250000
Epoch  600/10000 w:tensor([[ 37.9385],
        [-28.9494],
        [111.6601],
        [ 13.4636]], requires_grad=True) b

In [39]:
x_test = torch.FloatTensor(np.array([test_df['avgTemp'],test_df['minTemp'],test_df['maxTemp'],test_df['rainFall']]).T)
y_test = x_test.matmul(w)+ b
print(y_test)

tensor([[2831.9026],
        [1807.0104],
        [1813.9073],
        [3496.5962],
        [3333.3945],
        [3423.5461],
        [2954.6296],
        [4435.4043],
        [3648.7839],
        [4609.3984],
        [2082.8289],
        [3873.0354],
        [2948.1047],
        [3181.6604],
        [2435.9592],
        [3548.2068],
        [3471.8418],
        [4436.1060],
        [3123.7905],
        [3485.6868],
        [2437.1670],
        [2811.8064],
        [3217.7251],
        [1590.9962]], grad_fn=<AddBackward0>)


In [40]:
submit_df['Expected'] = y_test.detach().numpy()
submit_df.head()

,Id,Expected
0,0,2831.902588
1,1,1807.010376
2,2,1813.907349
3,3,3496.596191
4,4,3333.394531


In [30]:
submit_df.to_csv("./배추가격예측_data/submit_sample.csv",index=False)